In [1]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
%matplotlib inline

In [2]:
from numba import jit
# @jit()
def BF():
    global mat
    fnd = 0
    for i in xrange(1000000):
        # print i
        max_val = 20
        n, m = 10, 6
        mat = np.random.uniform( low=-max_val, high=max_val, size=n*m).astype(int).astype(float).reshape((n,m))
        try:
            _,_,_,_, info = pluq_ids(mat, debug=False)
        except:
            np.savetxt('except.txt', mat)
            print 'except'
            # return mat
        if info[1] > 1:
            np.savetxt('contra_{}_{}.txt'.format(fnd, info[1]), mat)
            return mat
            fnd += 1
            if fnd > 10:
                return
        

In [3]:
BF()

restored matrix
[[  1.70000000e+01   1.10000000e+01  -1.30000000e+01  -1.90000000e+01
   -5.00000000e+00   8.00000000e+00]
 [  0.00000000e+00  -2.72941176e+01  -4.70588235e-01   1.22352941e+01
    3.58823529e+00   5.88235294e-02]
 [  0.00000000e+00   0.00000000e+00   1.94289029e-16   1.70000000e+01
   -1.38750000e+01  -6.37500000e+00]
 [  0.00000000e+00   0.00000000e+00  -2.40689655e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.56206897e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   9.22413793e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.38103448e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   6.01724138e+00   0.00000000e+00
    0.00000000e+00   3.20000000e+01]
 [  0.00000000e+00   0.00000000e+00   8.68965517e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   

array([[-13.,   8.,  17., -19.,  -5.,  11.],
       [-15.,   9.,  19.,  -9.,  -2., -15.],
       [ -4.,  -4.,   5.,  16., -14.,  -7.],
       [-17.,  16.,  -9.,  12., -12.,   5.],
       [-11.,   8.,  -6., -13.,  -4.,  -2.],
       [ -1.,  -3.,  13.,  11.,  10.,  -8.],
       [ 13., -12.,   1., -11.,   6.,  -2.],
       [ 13.,  17.,  -9.,  11.,   7.,   0.],
       [ 16., -15.,  -9.,   4.,   9.,  19.],
       [  1., -19.,  10.,  19.,   9.,  10.]])

In [11]:
mat.astype(int)

array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]])

In [4]:
mat_pull_2 = np.array([[-13.,   8.,  17., -19.,  -5.,  11.],
       [-15.,   9.,  19.,  -9.,  -2., -15.],
       [ -4.,  -4.,   5.,  16., -14.,  -7.],
       [-17.,  16.,  -9.,  12., -12.,   5.],
       [-11.,   8.,  -6., -13.,  -4.,  -2.],
       [ -1.,  -3.,  13.,  11.,  10.,  -8.],
       [ 13., -12.,   1., -11.,   6.,  -2.],
       [ 13.,  17.,  -9.,  11.,   7.,   0.],
       [ 16., -15.,  -9.,   4.,   9.,  19.],
       [  1., -19.,  10.,  19.,   9.,  10.]])

In [5]:
_,_,_,_, info = pluq_ids(mat_pull_2, debug=False)
print info

restored matrix
[[  1.70000000e+01   1.10000000e+01  -1.30000000e+01  -1.90000000e+01
   -5.00000000e+00   8.00000000e+00]
 [  0.00000000e+00  -2.72941176e+01  -4.70588235e-01   1.22352941e+01
    3.58823529e+00   5.88235294e-02]
 [  0.00000000e+00   0.00000000e+00   1.94289029e-16   1.70000000e+01
   -1.38750000e+01  -6.37500000e+00]
 [  0.00000000e+00   0.00000000e+00  -2.40689655e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.56206897e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   9.22413793e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.38103448e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   6.01724138e+00   0.00000000e+00
    0.00000000e+00   3.20000000e+01]
 [  0.00000000e+00   0.00000000e+00   8.68965517e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   